In [ ]:
!pip install selenium
# !pip install chromedriver_autoinstaller

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

# from selenium.webdriver.chrome.options import Options
# import chromedriver_autoinstaller

In [22]:
# webdirver 설정, 빅카인즈 사이트 이동

driver = webdriver.Chrome('C:\chromedriver.exe')
driver.get('https://www.bigkinds.or.kr/v2/news/index.do')
driver.set_window_size(1280,1080)

In [23]:
# 검색기간 1주일로 설정

driver.find_element_by_xpath('//*[@id="collapse-step-1-body"]/div[3]/div/div[1]/div[1]/a').click()
time.sleep(1)
driver.find_element_by_xpath('//*[@id="srch-tab1"]/div/div[1]/span[2]').click()

In [24]:
# 검색창의 이름을 찾아서 사용자가 검색어 입력

querytxt = input('크롤링할 키워드는 무엇입니까?:')
searchbox = driver.find_element_by_id("total-search-key")
searchbox.send_keys(querytxt)
searchbox.send_keys("\n")  # 검색버튼 엔터

크롤링할 키워드는 무엇입니까?:넥슨


In [25]:
# 기사 분류 - 경제로 설정
# *주의* 화면을 크게해야 함. 경제 분류가 화면에 보여야 셀레니움이 잡을 수 있음

time.sleep(1)
driver.find_element_by_xpath('//*[@id="filterTab03"]/li[2]/span').click()

# from selenium.webdriver.common.keys import Keys
# .send_keys(Keys.ENTER)

In [26]:
# 기사 100건 씩 출력

driver.find_element_by_xpath('//*[@id="select2"]/option[4]').click()

In [27]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [28]:
# 전체 페이지 개수

pageNum = soup.select('div.lastNum')
totalPage = int(pageNum[0]['data-page'])
totalPage

1

In [29]:
contents = []
texts=[]
curPage = 1

while curPage <= totalPage:
    # bs4 초기화
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # 기사 리스트
    articles = soup.select('div.news-inner')
    # 페이지 번호 출력
    print('----- Current Page : {}'.format(curPage), '------')

    # 세부 데이터
    for article in articles:
        title = article.select_one('span.title-elipsis').text.strip()
        press =  article.select_one('div.info > div > a').text.strip()
        category = article.select_one('div.info > div > span.bullet-keyword').text.strip()
        date = article.select_one('div.info > p').text.strip()
        contents.append([title, press, category, date])
        
    # 기사 전문 
    for i in range(len(articles)):            
        driver.find_elements_by_css_selector('span.title-elipsis')[i].click()
        time.sleep(1)                
        text = driver.find_elements_by_css_selector('div.news-view-body')[0].text
        texts.append(text.replace('\n', ''))
        
        driver.find_element_by_xpath("//div[@id='news-detail-modal']/div/div/button").click()
        time.sleep(1)

            
    # 페이지 수 증가
    curPage += 1
    if curPage > totalPage:
        print('Crawling succeed')
        break
        
    # 페이지 이동 클릭
    driver.implicitly_wait(3)      
    nextbtn = driver.find_element_by_xpath('//*[@id="news-results-tab"]/div[6]/div[1]/div/div/div/div[4]/a')
    driver.execute_script("arguments[0].click();", nextbtn)
    
    # bs4 인스턴스 삭제
    del soup
    time.sleep(1.5)
 # 브라우저 종료
driver.close() 

len(contents)

----- Current Page : 1 ------
Crawling succeed


40

In [30]:
df = pd.DataFrame(data=contents, columns =['title', 'press', 'category', 'date'])
df['text'] = texts
# 파일 경로 지정
# df.to_excel("C:/Develops/save/1.xlsx", index=False)
df.to_excel("C:/Develops/save/" + querytxt +".xlsx", index=False)

## 클래스, 함수화

In [ ]:
class CRAWL:
    def __init__(self):
        self.options = Options()
        self.options.add_argument('headless'); # headless는 화면이나 페이지 이동을 표시하지 않고 동작하는 모드
        chromedriver_autoinstaller.install()
        self.driver = webdriver.Chrome(options=self.options) # 브라우저 창 안보이기
        self.totalPage = 1 # 전체 페이지수
        
        
    def getBaseURL(self,url):
        self.driver.implicitly_wait(5)  # 크롬 브라우저 내부 대기
        self.driver.set_window_size(1920,1280) # 브라우저 사이즈
        self.driver.get(url)
        
        # 검색기간 1주일로 설정
        driver.find_element_by_xpath('//*[@id="collapse-step-1-body"]/div[3]/div/div[1]/div[1]/a').click()
        time.sleep(1)
        driver.find_element_by_xpath('//*[@id="srch-tab1"]/div/div[1]/span[2]').click()
        

    def getTotalPage(self,url):
        self.getBaseURL(url)
        
        # 검색창의 이름을 찾아서 사용자가 검색어 입력
        # query_txt = input('크롤링할 키워드는 무엇입니까?:')
        searchbox = driver.find_element_by_id("total-search-key")
        searchbox.send_keys("오리온")  
        searchbox.send_keys("\n")  # 검색버튼 엔터
        
        # 경제 기사만 선택
        time.sleep(1)
        driver.find_element_by_xpath('//*[@id="filterTab03"]/li[2]/span').click()

        soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        
        pageNum = soup.select('div.lastNum')
        self.totalPage  = int(pageNum[0]['data-page'])
        
        del soup
        
    
    def crawling(self,url):
        self.getTotalPage(url)
        totalPage = int(self.totalPage)  # 크롤링할 전체 페이지수
        print('total Pages : ',totalPage)
        curPage = 1  # 현재 페이지
        
        contents = []
        texts = []
        while curPage <= totalPage:
            # bs4 초기화
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            # 기사 리스트
            articles = soup.select('div.news-inner')
            # 페이지 번호 출력
            print('----- Current Page : {}'.format(curPage), '------')

            # 세부 데이터
            for article in articles:
                title = article.select_one('span.title-elipsis').text.strip()
                press =  article.select_one('div.info > div > a').text.strip()
                category = article.select_one('div.info > div > span.bullet-keyword').text.strip()
                date = article.select_one('div.info > p').text.strip()
                
                contents.append([title, press, category, date])
            
            # 기사 전문 
#             for i in range(0, 100):
#                 driver.find_elements_by_css_selector('span.title-elipsis')[i].click()
#                 time.sleep(1)
#                 text = driver.find_elements_by_css_selector('div.news-view-body')[0].text
#                 texts.append(text.replace('\n', ''))
#                 driver.find_element_by_xpath("//div[@id='news-detail-modal']/div/div/button").click()

            # 페이지 수 증가
            curPage += 1
            if curPage > totalPage:
                print('Crawling succeed')
                break

            # 페이지 이동 클릭
            driver.implicitly_wait(3)      
            nextbtn = driver.find_element_by_xpath('//*[@id="news-results-tab"]/div[6]/div[1]/div/div/div/div[4]/a')
            driver.execute_script("arguments[0].click();", nextbtn)

            # bs4 인스턴스 삭제
            del soup
            time.sleep(2)
            
            
    def closing(self):    
        self.driver.close()  # 브라우저 종료  
    
    
    
if __name__ == "__main__":
    baseURL = 'https://www.bigkinds.or.kr/v2/news/index.do'
    crawl = CRAWL()
    crawl.crawling(baseURL)
    crawl.closing()


+ 최근 1주일
1. 빅카인즈 경제 기사 키워드당 100개만 기져오기
2. 매일경제 + 한국경제 기사 가져오기

In [ ]:
# [제목, 언론사, 분류, 날짜, 기자이름, 기사내용]

# contents = [] 
# for article in articles:
#     title = article.select('span.title-elipsis')
#     press =  article.select('div.info > div > a')
#     category = article.select('div.info > div > span.bullet-keyword')
#     date = article.select('div.info > p')
#     contents.append([title[0].text.strip(), press[0].text.strip(), category[0].text.strip(), date[0].text.strip(), date[1].text.strip()])

# len(contents)

In [ ]:
# 기사 전문 

# texts = []
# for i in range(0, 100):
#     driver.find_elements_by_css_selector('span.title-elipsis')[i].click()
#     time.sleep(1)
#     text = driver.find_elements_by_css_selector('div.news-view-body')[0].text
#     texts.append(text.replace('\n', ''))
#     driver.find_element_by_xpath("//div[@id='news-detail-modal']/div/div/button").click()

# len(texts)

In [ ]:
# 다음 페이지 넘기기

# nextbtn = driver.find_element_by_xpath('//*[@id="news-results-tab"]/div[6]/div[1]/div/div/div/div[4]/a')
# browser.execute_script("arguments[0].click();", nextbtn)
